In [11]:
from docplex.cp.model import *
from docplex.cp.config import get_default

In [12]:
from config import setup
setup()

In [13]:
n = 4

F = ['f1', 'f2', 'f3', 'f4']
H = ['h1', 'h2', 'h3', 'h4']

L = {
    "f1":['h2', 'h1', 'h3', 'h4'],
    "f2":['h3', 'h4', 'h1', 'h2'],
    "f3":['h1', 'h3', 'h4', 'h2'],
    "f4":['h2', 'h1', 'h3', 'h4'],

    "h1":['f2', 'f3', 'f1', 'f4'],
    "h2":['f4', 'f1', 'f3', 'f2'],
    "h3":['f2', 'f4', 'f1', 'f3'],
    "h4":['f3', 'f1', 'f4', 'f2']
}


In [14]:
mdl = CpoModel(name="Mariages")

#Lpos[h,f] postition de f dans L(h) + position de h dans L(h)
LPos = {}
for h in H:
    for f in F:
        LPos[h,f]=L[h].index(f)
        LPos[f,h]=L[f].index(h)


In [15]:
#x[h,f] mariage entre h et f
x = {}
z = {}
for h in H:
    for f in F:
        x[h,f]=mdl.binary_var(name="x_" + h + "_" + f)
        z[h,f]=mdl.binary_var(name="z_" + h + "_" + f)

#y[i] cout du marriage de i (somme des couts du mariage pour les 2 partis) 
y = {}
for i in H+F:
    y[i]=mdl.integer_var(0,2*(n-1),name="y_"+i)




In [16]:
#Ajout des contraintes

#Contrainte de monogamie
for h in H:
    mdl.add(sum([x[h,f] for f in F])==1)
for f in F:
    mdl.add(sum([x[h,f] for h in H])==1)


#Valeur de y:
for h in H:
    mdl.add(y[h]==sum([(x[h,f]*LPos[h,f]) for f in F]))
for f in F:
    mdl.add(y[f]==sum([(x[h,f]*LPos[f,h]) for h in H]))

#On cherches à minimiser les couts des marriages
#dl.add(minimize(max([y[i] for i in H+F])))

In [17]:
#Couple bloquant =
#si (h,f) est un couple bloquant si 
#   - le partenaire actuel de h et en plus haute postion de preference que f
#   - et le partenaire actuel de f et en plus haute postion de preference que h
for h in H:
    for f in F:
        mdl.add(mdl.if_then((y[h]>LPos[h,f]) & (y[f]>LPos[f,h]), z[h,f] == 1))

mdl.add(sum(z[h,f] for h in H for f in F)==0)

In [18]:
lsol = mdl.start_search(SearchType="DepthFirst",trace_log=False)


In [19]:
#Affichage des solutions:
for solution in lsol:
    print("\n NOUVELLE SOLUTION :")
    for h in H:
        for f in F:
            if(solution[x[h,f]]==1):
                print("Mariage entre ",h," et ",f," (Couts = ",LPos[h,f],LPos[f,h],")")



 NOUVELLE SOLUTION :
Mariage entre  h1  et  f3  (Couts =  1 0 )
Mariage entre  h2  et  f4  (Couts =  0 0 )
Mariage entre  h3  et  f2  (Couts =  0 0 )
Mariage entre  h4  et  f1  (Couts =  1 3 )
